<a href="https://colab.research.google.com/github/jakes1811/Smart-Waste-Segregation-System/blob/main/Smart_Waste_Segregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os

# Path to your zip file
zip_file_path = "/content/Waste.zip"

# Extract to this folder
extract_folder = "/content/Waste"

# Create the directory if it doesn’t exist
os.makedirs(extract_folder, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Extraction completed! Files are in:", extract_folder)


Extraction completed! Files are in: /content/Waste


In [ ]:
import cv2
import os

img_size = (224, 224)
dataset_path = "/content/Waste"

for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    # Check if it is a directory
    if os.path.isdir(category_path):
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            # Check if it is an image file
            if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img = cv2.imread(img_path)
                if img is not None: # Check if image was read successfully
                    img = cv2.resize(img, img_size)
                    cv2.imwrite(img_path, img)  # Save resized images
                else:
                    print(f"Could not read image: {img_path}")
    else:
        print(f"Skipping non-directory: {category_path}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load MobileNetV2 as feature extractor
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)
model.save('waste_classifier_mobile.h5')

